In [1]:
import os

os.environ["LANGCHAIN_PROJECT"] = "demo-optimization"
os.environ["LANGCHAIN_API_KEY"] = "..."

In [2]:
from langsmith import Client

client = Client()

In [6]:
client.request_with_retries(
    "POST",
    client.api_url + "/runs/rules",
    {
        "json": {
            "display_name": "queries",
            "session_id": "942208fa-21ee-4ed8-ab5d-6ed5a9090cdf",
            "sampling_rate": 1,
            "filter": 'eq(name, "generate_movie_search")',
            "trace_filter": 'and(eq(feedback_key, "correctness"), eq(feedback_score, 1))',
            "add_to_dataset_id": "ef2d35f5-bb69-41e0-bf91-c5b5eb631d43",
        },
        "headers": client._headers,
    },
)

<Response [200]>

In [7]:
from openai import OpenAI
from langsmith import wrappers, traceable

openai = wrappers.wrap_openai(OpenAI())

In [8]:
import json

tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_movies",
            "description": "Retrieve a list of relevant movies and their metadata from a movie database.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query used to retrieve movies from the movie database, for example 'Christopher Nolan films'",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

system_prompt = """
Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.
Note that if the question does not require additional search and can be answered using the chat history, simply respond with the answer.
Don't make up content that's not supplied in chat history.
"""


@traceable
def generate_movie_search(messages):
    messages = [
        {"role": "system", "content": system_prompt},
    ] + messages
    result = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo-0613", tools=tools
    )
    return result.choices[0].message


def _convert_docs(results):
    return [
        {
            "page_content": r,
            "type": "Document",
        }
        for r in results
    ]


@traceable(run_type="retriever")
def retrieve_movies(query):
    # Foo retriever. In production, this would search an actual database
    if "family-friendly" in query.lower():
        return _convert_docs(["Lion King", "Finding Nemo", "The Incredibles"])
    elif "sci-fi" in query.lower():
        return _convert_docs(["Blade Runner 2049", "Interstellar", "The Martian"])
    elif "nature" in query.lower():
        return _convert_docs(["Planet Earth II", "Blue Planet II", "Our Planet"])
    elif "christopher nolan" in query.lower():
        return _convert_docs(["Inception", "Dunkirk", "Interstellar"])
    else:
        return _convert_docs(
            ["Crazy Rich Asians", "The Big Sick", "When Harry Met Sally"]
        )


@traceable
def execute_function_call(message):
    if message.tool_calls[0].function.name == "retrieve_movies":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = retrieve_movies(query)
    else:
        results = (
            f"Error: function {message.tool_calls[0].function.name} does not exist"
        )
    return results


@traceable
def generate_answer(messages, context):
    messages = [
        {
            "role": "system",
            "content": f"Answer the user's question based only on the content below:\n\n{context}",
        },
    ] + messages
    result = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo", temperature=0
    )
    return result.choices[0].message.content


@traceable
def rag_pipeline(messages):
    message = generate_movie_search(messages)
    if message.tool_calls is None:
        return message.content
    else:
        docs = execute_function_call(message)
        context = "\n".join([doc["page_content"] for doc in docs])
        return generate_answer(messages, context)

In [9]:
rag_pipeline([{"role": "user", "content": "what are some family movies to watch?"}])

'While "Crazy Rich Asians," "The Big Sick," and "When Harry Met Sally" are all great movies, they are not typically considered family movies. For family-friendly options, you may want to consider films like "Finding Nemo," "Toy Story," or "The Incredibles."'

In [12]:
rag_pipeline([{"role": "user", "content": "what are some movies about aliens?"}])

"I'm sorry, but the movies you mentioned - Crazy Rich Asians, The Big Sick, and When Harry Met Sally - do not involve aliens. If you're looking for movies about aliens, some popular options include E.T. the Extra-Terrestrial, Independence Day, and Arrival."

In [14]:
rag_pipeline([{"role": "user", "content": "Need a light-hearted movie"}])

'I recommend watching "When Harry Met Sally." It is a classic romantic comedy that is both heartwarming and funny. Enjoy!'

In [15]:
examples = list(client.list_examples(dataset_name="demo-optimization"))

In [17]:
examples[0].inputs

{'messages': [{'role': 'user',
   'content': 'what are some movies about aliens?'}]}

In [18]:
examples[0].outputs

{'output': {'role': 'assistant',
  'tool_calls': [{'id': 'call_pmNG2wQiXMTy77aogKyYkpc0',
    'type': 'function',
    'function': {'name': 'retrieve_movies',
     'arguments': '{\n"query": "movies about aliens"\n}'}}]}}

In [33]:
few_shot_examples = []
for example in examples:
    few_shot_examples.extend(example.inputs["messages"])
    few_shot_examples.append(example.outputs["output"])
    few_shot_examples.extend(
        [
            {"role": "tool", "tool_call_id": m["id"], "content": "..."}
            for m in example.outputs["output"]["tool_calls"]
        ]
    )

In [34]:
@traceable
def generate_movie_search(messages):
    messages = (
        [
            {"role": "system", "content": system_prompt},
        ]
        + few_shot_examples
        + messages
    )
    result = openai.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo-0613", tools=tools
    )
    return result.choices[0].message

In [35]:
@traceable
def rag_pipeline(messages):
    message = generate_movie_search(messages)
    if message.tool_calls is None:
        return message.content
    else:
        docs = execute_function_call(message)
        context = "\n".join([doc["page_content"] for doc in docs])
        return generate_answer(messages, context)

In [37]:
rag_pipeline([{"role": "user", "content": "Suggest a suspenseful movie"}])

'If you enjoyed "Crazy Rich Asians," "The Big Sick," and "When Harry Met Sally," you might like the suspenseful movie "Gone Girl."'